In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import pickle
import csv
import torch
import torchvision
from sklearn import preprocessing
from torchvision import transforms, datasets
from tqdm import tqdm
from tensorflow import keras
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D,LeakyReLU
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D ,BatchNormalization,Activation
from tensorflow.keras.optimizers import Adam ,RMSprop
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.regularizers import l2
from tensorflow.keras import layers

In [2]:
SUBMISSION_HEADER = ['Id', 'class']
RANDOM_STATE = 0

In [3]:
def open_pickled_file(file):
    with open(file, 'rb') as f:
        data = pd.read_pickle(file)
    return np.asarray(data)

In [80]:
def export_predictions(y_pred):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    filename = '../prediction-{}.csv'.format(timestr)
    print(filename)
    file = open(filename, 'w+', newline ='')
    with file:    
        writer = csv.writer(file)
        writer.writerow(SUBMISSION_HEADER)
        writer.writerows(enumerate(y_pred))

In [5]:
def preprocess(X, y):
    X = scale(X)
    y = encode(y)
    return X, y

In [6]:
def encode(y):
    le = preprocessing.LabelEncoder()
    le.fit(y)
    return le.transform(y)

In [7]:
def scale(X):
    max_val = np.max(X)
    return X/max_val

In [10]:
%cd ~/git/map-kaggle/
X_train_raw = open_pickled_file('./dataset/x_train.pkl.zip')
y_train_raw = open_pickled_file('./dataset/y_train.pkl')
X_test_raw = open_pickled_file('./dataset/x_test.pkl.zip')

/Users/athul/git/map-kaggle


In [91]:
X_train, y_train = preprocess(X_train_raw, y_train_raw)

In [92]:
X_test = X_test_raw/255
X_test = X_test.reshape(-1, 96, 96, 1)

In [93]:
Y_enc = to_categorical(y_train, 11)

In [96]:
np.unique(y_train)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [48]:
X_train_shape = X_train.reshape(-1, 96, 96, 1)

In [58]:
X_train, X_val, y_train, y_val = train_test_split(X_train_shape, Y_enc, test_size=0.2, random_state=0)

In [59]:
batch_size = 128
epochs = 5
num_classes = 11

In [60]:
model = Sequential([
# data_augmentation,
layers.Conv2D(32, kernel_size=(3, 3), activation='linear', input_shape=(96,96,1), padding='same'),
layers.LeakyReLU(alpha=0.1),
layers.MaxPooling2D((2, 2), padding='same'),

layers.Conv2D(64, (3, 3), activation='linear', padding='same'),
layers.LeakyReLU(alpha=0.1),
layers.MaxPooling2D(pool_size=(2, 2), padding='same'),

layers.Conv2D(128, (3, 3), activation='linear',padding='same'),
layers.LeakyReLU(alpha=0.1),          
layers.MaxPooling2D(pool_size=(2, 2),padding='same'),

layers.Conv2D(256, (3, 3), activation='linear',padding='same'),
layers.LeakyReLU(alpha=0.1),                 
layers.MaxPooling2D(pool_size=(2, 2),padding='same'),
layers.Dropout(0.25),

layers.Flatten(),

layers.Dense(128, activation='linear'),
layers.LeakyReLU(alpha=0.1),  
layers.Dropout(0.25),

layers.Dense(num_classes, activation='softmax')])

In [61]:
def data_augmentation(X, y):
    img = ImageDataGenerator(rotation_range = 90)
    Train_batch = img.flow(X, y, batch_size = 128)
    return Train_batch

In [98]:
Y_enc

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [62]:
dataload = data_augmentation(X_train_shape,Y_enc)

In [63]:
tf.test.is_built_with_cuda()

False

In [64]:
tf.test.gpu_device_name()

2021-11-28 23:13:10.940609: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-28 23:13:10.940702: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


'/device:GPU:0'

In [75]:
%%timeit -n1 -r1 
with tf.device('/GPU:0'):
    model.compile(loss="categorical_crossentropy", optimizer= Adam(learning_rate=0.001), metrics=["accuracy"])
    # This model is overfitting
    # model.fit(X_train, y_train, epochs=50, shuffle= True)
    model.fit(dataload, batch_size=batch_size, epochs=100, shuffle=True)

Epoch 1/100


2021-11-28 23:15:35.778134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


93/93 [==============================] - 18s 180ms/step - loss: 1.9513 - accuracy: 0.3352
Epoch 2/100
93/93 [==============================] - 19s 201ms/step - loss: 1.7680 - accuracy: 0.3989
Epoch 3/100
93/93 [==============================] - 17s 180ms/step - loss: 1.6309 - accuracy: 0.4485
Epoch 4/100
93/93 [==============================] - 16s 169ms/step - loss: 1.5601 - accuracy: 0.4686
Epoch 5/100
93/93 [==============================] - 20s 219ms/step - loss: 1.4690 - accuracy: 0.5105
Epoch 6/100
93/93 [==============================] - 23s 253ms/step - loss: 1.4145 - accuracy: 0.5229
Epoch 7/100
93/93 [==============================] - 20s 217ms/step - loss: 1.3623 - accuracy: 0.5410
Epoch 8/100
93/93 [==============================] - 15s 155ms/step - loss: 1.3153 - accuracy: 0.5544
Epoch 9/100
93/93 [==============================] - 14s 146ms/step - loss: 1.2597 - accuracy: 0.5800
Epoch 10/100
93/93 [==============================] - 13s 144ms/step - loss: 1.2337 - accuracy

In [76]:
model.evaluate(X_val, y_val)

2021-11-28 23:41:58.349275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


75/75 [==============================] - 2s 15ms/step - loss: 0.3828 - accuracy: 0.8688


[0.38277286291122437, 0.8687973022460938]

In [77]:
pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)
export_predictions(pred)